# The Lasso

The lasso is a shrinkage method like ridge with one major difference: the penalty term. In ridge regression the penalty term is defined as $\lambda\sum_{j=1}^p \beta_j^2$, while in lasso it's defined as $\lambda\sum_{j=1}^p |\beta_j|$. This later penalty makes the solution nonlinear and there is no closed-form solution as in ridge regression.

The lasso solution is a quadratic programming problem. However, as we can see in section 3.4.4, we can use *LARS* with a slight modification to obtain lasso solution.

In this notebook, we try to implement the lasso-modified LARS from scratch, and to do that we need to understand how the original *LARS* algorithm works:

   * Initialization
       * First, we need to standarize all the predictors $\mathbf{X}$ to have a zero mean and unit variance, and set all the initial coefficients to zero $\beta_1 = \beta_2 = \cdots = \beta_p = 0$. The initial residual is $r = y - \bar y$
       * Find the feature $j$ that has the highest absolute correlation with the residue $r$
       $$j = \underset{j}{\text{argmax }} |\mathbf{X}^T r|$$
       * Add $j$ to the active set $A$

   * For every step $k$
       * Move $\beta[A]$ in the direction of:
       $$\delta_k = \left(\mathbf{X}[A_k]^\top\mathbf{X}[A_k]\right)^{-1}\mathbf{X}[A_k]^\top r$$
       
       The coefficient profile evolves as:
       $$\beta[A_k](\alpha) = \beta[A_k] + \alpha \delta_k$$



In [169]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import itertools
import sys
import scipy.stats
from matplotlib.ticker import FormatStrFormatter
from ml_datasets.esl import ProstateCancer
from esl.utils import scale

In [170]:
prostate_cancer = ProstateCancer(verbose=0)

In [171]:
df = prostate_cancer.load()
# extract input and output dataframe
train_test = prostate_cancer.train_test
df_x = df[prostate_cancer.meta[:-1]]
df_y = df[prostate_cancer.meta[-1]]

x_train = scale(df_x[train_test=='T'].copy()).values
y_train =  df_y[train_test=='T'].copy().values

In [172]:
def kfold(x, y, nfold=10):
    num_data = len(y)
    index = np.arange(num_data)
    np.random.seed(2)
    np.random.shuffle(index)
    
    num_data = len(y)
    avg = len(index) / float(nfold)
    last = 0.0
    
    x_dict = dict()
    y_dict = dict()
    
    i = 0
    while last < num_data:
        index_val = index[int(last):int(last + avg)]
#         print(last, avg, int(last + avg))
        index_train = np.array([j for j in range(num_data) if j not in index_val])
        
        y_dict[9 - i] = {'train': y[index_train], 'val': y[index_val]}
        x_dict[9 - i] = {'train': x[index_train, :], 'val': x[index_val, :]}
        
        last += avg
        i += 1
    
    return x_dict, y_dict

In [173]:
# x_dict, y_dict = kfold(x_train, y_train, nfold=10)

In [189]:
n, p = x_train.shape

# get initial residual by subtracting the mean of y_train from y_train
res =  y_train - np.mean(y_train)
x_intercept = np.ones_like(x_train[:, 0]).reshape(-1, 1)

intercept = np.linalg.inv(x_intercept.T @ x_intercept) @ x_intercept.T @ y_train

# initial correlation
x_train_ = np.hstack([x_intercept, x_train])
corr = x_train_[:, 1:].T @ residual
best_index = np.argmax(np.abs(corr)) + 1

# initialize active set A
A = [0, best_index]
# A.add(best_index)
print(A)
beta = np.zeros(p + 1)
beta[0] = intercept

beta[A] = np.linalg.inv(x_train_[:, A].T @ x_train_[:, A]) @ x_train_[:, A].T @ y_train
print(beta)
# # initialize sign array sj
# sign = np.zeros(p)
# # sign[list(A)] = np.sign(corr[list(A)])

# mu_hat_A = x_train[:, list(A)] @ beta[list(A)]

# for k in range(1, p + 1):
#     # residue at step k on the active set A
#     res = y_train - mu_hat_A
    
#     corr = x_train.T @ res
#     corr_best_index = np.argmax(np.abs(corr))
    
#     corr_best = corr[corr_best_index]
#     sign[list(A)] = np.sign(corr[list(A)])
    
#     X_A = np.multiply(sign, x_train)[:, list(A)]
#     I_A = np.ones(len(A))
#     G_A = np.array(X_A.T @  X_A)
#     A_A = np.sqrt(1 / (I_A.T @ np.linalg.inv(G_A) @ I_A))

#     w_A = A_A * np.linalg.inv(G_A) @ I_A
    
#     # equiangular vector
#     u_A = X_A @ w_A 
    
#     # angle
#     a = x_train.T @ u_A
#     print(a)
#     # this loop  finds gamma

#     # if not in the last step
#     if k < p:
#         gamma = np.inf
#         for j in np.arange(p):

#             # this is because we want j in A-complement, or j not in A
#             if j in list(A):
#                 continue

#             left = (corr_best - corr[j]) / (A_A - a[j])
#             if left < 0:
#                 left = np.inf

#             right = (corr_best + corr[j]) / (A_A + a[j])
#             if right < 0:
#                 right = np.inf

#             gamma_temp = np.min([left, right])
#             if gamma_temp < gamma:
#                 gamma = gamma_temp
#                 best_j = j
                
#     else:
#         gamma = corr_best / A_A

#     beta[list(A)] = beta[list(A)] + gamma * np.linalg.inv(X_A.T @ X_A) @ X_A.T @ res
# #     mu_hat_A = mu_hat_A + gamma * u_A

# print(beta)
    

[0, 1]
[2.45234509 0.87888041 0.         0.         0.         0.
 0.         0.         0.        ]


In [36]:
for k in np.arange(p):
    residual = y_train - y_hat
    
    mse = np.mean(residual.T @ residual)
    
    pred = x_train @ beta
    
    corr = x_train.T @ residual
    
    Xa = X[:, list(active_set)]
    Xa *= sing[list(active_set)]
    
    Ga = Xa.T @ Xa
    Ga = Xa.T @ Xa
    Ga_inv = np.linalg(Ga)
    Ga_inv_red_cols = np.sum(Ga_inv, 1)
    
    Aa = 1 / np.sqrst(np.sum(Ga_inv_red_cols))
    omega = Aa * Ga_inv_red_cols
    
    equiangular = Xa @ omega
    
    cos_angle = x_train.T @ equiangular
    gamma = 0
    
    
    
    
    
    
    corr_best_idx = np.argmax(corr)
    A[:, corr_best_idx] = x_train[:, corr_best_idx]
    
    delta[corr_best_idx] = A[:, corr_best_idx].T @ r
    
    beta_hat[corr_best_idx]  = beta_hat[corr_best_idx]  + alpha * delta[corr_best_idx] 

    f = A[:, corr_best_idx] * beta_hat[corr_best_idx]
    
    r = r - f
    
    u = A[:, corr_best_idx] * delta[corr_best_idx]
    f = f + alpha * u

In [34]:
print(beta_hat)

[ 1.11881477e+02 -1.16249216e+17  0.00000000e+00  3.74581278e+05
  2.26675088e+05  0.00000000e+00  5.87804135e+16  0.00000000e+00]
